In [8]:
import cv2
import numpy as np
import glob
from skimage.util import img_as_float
import pandas as pd
import os

from matplotlib import pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

In [10]:
os.chdir("../../data/phase-02/sugarcanetiles")
#!ls
files = glob.glob('*-*-B08*.png')
files[:10]

['1536-1024-B08-2016-12-22.png',
 '1536-1024-B08-2017-01-01.png',
 '1536-1024-B08-2017-01-11.png',
 '1536-1024-B08-2017-02-10.png',
 '1536-1024-B08-2017-02-20.png',
 '1536-1024-B08-2017-03-02.png',
 '1536-1024-B08-2017-03-12.png',
 '1536-1024-B08-2017-04-01.png',
 '1536-1024-B08-2017-04-11.png',
 '1536-1024-B08-2017-05-01.png']

In [69]:
band8_files = glob.glob('*-*-B08*.png')
band4_files = glob.glob('*-*-B04*.png')
band8_files.sort()
band4_files.sort()

In [70]:
len(band8_files)
len(band4_files)

4615

In [71]:
#65 * 721 = 4615
65*71

4615

In [72]:
tiles = pd.read_csv("../../1_raw/tiles.csv")
tiles

,tile,N_dates,tile_x,tile_y
0,1536_1024,71,1536,1024
1,1536_1536,71,1536,1536
2,2048_1024,71,2048,1024
3,2048_1536,71,2048,1536
4,4608_4608,71,4608,4608
...,...,...,...,...
60,7680_5120,71,7680,5120
61,7680_5632,71,7680,5632
62,7680_9728,71,7680,9728
63,8192_10240,71,8192,10240


In [73]:
dates = pd.read_csv("../../1_raw/dates.csv")
dates

,date,N_tiles
0,2016-12-22,65
1,2017-01-01,65
2,2017-01-11,65
3,2017-02-10,65
4,2017-02-20,65
...,...,...
66,2019-06-20,65
67,2019-06-30,65
68,2019-07-20,65
69,2019-07-30,65


In [74]:
def gen_ndvi(band8_file, band4_file):
    ndvi_mean=[]
    dates= []
    for band4_file,band8_file in zip(band4_files, band8_files):
        dates.append(band4_file[-14:-4])
    
        band4 = cv2.imread(band4_file)
        band8 = cv2.imread(band8_file)
    
        ndvi = (band8.astype(float) - band4.astype(float))/(band8.astype(float)+band4.astype(float))

        image = img_as_float(ndvi)
        image = image[:, :, ::-1]
    
        ndvi_mean.append(ndvi.mean())

    df = pd.DataFrame([dates, ndvi_mean]).T
    df.columns = ['dates', 'ndvi']
    return(df)

In [75]:
band8_files = glob.glob('../../phase-01/data/sentinel-2a-tile-7680x-10240y/timeseries/7680-10240-B08*.png')
band4_files = glob.glob('../../phase-01/data/sentinel-2a-tile-7680x-10240y/timeseries/7680-10240-B04*.png')
band8_files.sort()
band4_files.sort()

In [76]:
def get_band(x, y, b):
    band_files = glob.glob('%s-%s-B0%s-*.png' % (x, y, b))
    band_files.sort()
    return(band_files)

In [77]:
tile = tiles.iloc[0,:]
x = tile['tile_x']
y = tile['tile_y']
band8_files = get_band(x, y, 8)
band4_files = get_band(x, y, 4)
df = gen_ndvi(band8_files, band4_files)
df.iloc[0,:]
#def 
#band8_files = get_band(x['tile_x'], x['tile_y'], 8), band4_files =
#tiles.apply(lambda x:  )

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


dates    2016-12-22
ndvi       0.337714
Name: 0, dtype: object

In [78]:
def get_df_from_tile(tile):
   x = tile['tile_x']
   y = tile['tile_y']
   band8_files = get_band(x, y, 8)
   band4_files = get_band(x, y, 4)
   #print(band8_files)
   #print(band4_files)
   ndvi_df = gen_ndvi(band8_files, band4_files)
   ndvi_df['x'] = x
   ndvi_df['y'] = y
   print(x, y)
   ndvi_df.iloc[0,:]
   return(ndvi_df)

In [79]:
tile = tiles.iloc[0,:]
ndvi_df = get_df_from_tile(tile)
df = ndvi_df 


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


1536 1024


In [80]:
df

,dates,ndvi,x,y
0,2016-12-22,0.337714,1536,1024
1,2017-01-01,0.187798,1536,1024
2,2017-01-11,0.375741,1536,1024
3,2017-02-10,0.512083,1536,1024
4,2017-02-20,0.147082,1536,1024
...,...,...,...,...
66,2019-06-20,0.380908,1536,1024
67,2019-06-30,0.358928,1536,1024
68,2019-07-20,0.360396,1536,1024
69,2019-07-30,0.323928,1536,1024


In [81]:
for i in range(1, tiles.shape[0]):
    print(i)
    tile = tiles.iloc[i,:]
    ndvi_df = get_df_from_tile(tile)
    df = df.append(ndvi_df)

1


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


1536 1536
2
2048 1024
3
2048 1536
4
4608 4608
5
4608 5120
6
4608 5632
7
4608 6144
8
5120 3584
9
5120 4096
10
5120 4608
11
5120 5120
12
5120 5632
13
5120 6144
14
5120 6656
15
5120 7168
16
5632 4096
17
5632 4608
18
5632 5120
19
5632 5632
20
5632 6144
21
5632 6656
22
5632 7168
23
5632 7680
24
5632 8192
25
5632 8704
26
6144 4096
27
6144 4608
28
6144 5120
29
6144 5632
30
6144 6144
31
6144 6656
32
6144 7168
33
6144 7680
34
6144 8192
35
6144 8704
36
6144 9216
37
6144 9728
38
6656 10240
39
6656 4096
40
6656 4608
41
6656 5120
42
6656 5632
43
6656 6144
44
6656 6656
45
6656 7168
46
6656 7680
47
6656 8192
48
6656 8704
49
6656 9216
50
6656 9728
51
7168 10240
52
7168 4096
53
7168 4608
54
7168 5120
55
7168 5632
56
7168 6656
57
7168 7168
58
7168 9728
59
7680 10240
60
7680 5120
61
7680 5632
62
7680 9728
63
8192 10240
64
8704 10240


In [82]:
df.head()

,dates,ndvi,x,y
0,2016-12-22,0.337714,1536,1024
1,2017-01-01,0.187798,1536,1024
2,2017-01-11,0.375741,1536,1024
3,2017-02-10,0.512083,1536,1024
4,2017-02-20,0.147082,1536,1024


In [83]:
df.shape

(4615, 4)

In [84]:
df.tail()

,dates,ndvi,x,y
66,2019-06-20,0.380908,8704,10240
67,2019-06-30,0.358928,8704,10240
68,2019-07-20,0.360396,8704,10240
69,2019-07-30,0.323928,8704,10240
70,2019-08-09,0.326491,8704,10240


In [85]:
ndvi_df

,dates,ndvi,x,y
0,2016-12-22,0.337714,8704,10240
1,2017-01-01,0.187798,8704,10240
2,2017-01-11,0.375741,8704,10240
3,2017-02-10,0.512083,8704,10240
4,2017-02-20,0.147082,8704,10240
...,...,...,...,...
66,2019-06-20,0.380908,8704,10240
67,2019-06-30,0.358928,8704,10240
68,2019-07-20,0.360396,8704,10240
69,2019-07-30,0.323928,8704,10240


In [86]:
df.append(ndvi_df)

,dates,ndvi,x,y
0,2016-12-22,0.337714,1536,1024
1,2017-01-01,0.187798,1536,1024
2,2017-01-11,0.375741,1536,1024
3,2017-02-10,0.512083,1536,1024
4,2017-02-20,0.147082,1536,1024
...,...,...,...,...
66,2019-06-20,0.380908,8704,10240
67,2019-06-30,0.358928,8704,10240
68,2019-07-20,0.360396,8704,10240
69,2019-07-30,0.323928,8704,10240


In [87]:
df.to_csv("../../2_processed/phase-02_ndvi.csv", header=True, index=False)